In [1]:
%load_ext autoreload
%autoreload 2

# Imports and definitions

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr

In [3]:
from ecephys.data import channel_groups, paths
from ecephys.sglx_utils import load_timeseries
from ecephys.signal.timefrequency import parallel_spectrogram_welch

In [4]:
DELTA_RANGE = (0.5, 4)
THETA_RANGE = (5, 10)

In [5]:
def get_spectrogram(sig):
    freqs, spg_times, spg = parallel_spectrogram_welch(sig.values, sig.fs)
    spg_times = spg_times + sig.time.values.min()
    spg = xr.DataArray(spg, dims=("frequency", "time", "channel"), coords={"frequency": freqs, "time": spg_times, "channel": sig.channel.values})
    return spg.median(dim="channel")

In [6]:
def _get_bandpower(spg, f_range, chans):
    bandpower = spg.sel(frequency=slice(*f_range)).sum(dim="frequency")
    bandpower.attrs['long_name'] = "{} Hz PSD".format(f_range)
    bandpower.attrs['units'] = 'uV^2/Hz'
    bandpower.attrs['chans'] = chans
    bandpower.attrs['freq_range'] = f_range
    
    return bandpower

In [7]:
def run_bandpower_pipeline_on_file(bin_path, bandpower_path, cx_chans, sr_chans):
    cx_sig = load_timeseries(bin_path, cx_chans, xarray=True)
    cx_spg = get_spectrogram(cx_sig)
    cx_theta = _get_bandpower(cx_spg, THETA_RANGE, cx_chans)
    cx_delta = _get_bandpower(cx_spg, DELTA_RANGE, cx_chans)
    
    sr_sig = load_timeseries(bin_path, sr_chans, xarray=True)
    sr_spg = get_spectrogram(sr_sig)
    sr_theta = _get_bandpower(sr_spg, THETA_RANGE, sr_chans)
    sr_delta = _get_bandpower(sr_spg, DELTA_RANGE, sr_chans)
    
    bandpower = xr.Dataset({'mpta_delta': cx_delta, 'mpta_theta': cx_theta, 'sr_delta': sr_delta, 'sr_theta': sr_theta})
    bandpower.to_netcdf(bandpower_path)
    bandpower.close()

In [8]:
def run_bandpower_pipeline_on_condition(subject, condition):
    cx_chans = channel_groups.superficial_ctx[subject]
    sr_chans = channel_groups.stratum_radiatum_140um_to_200um[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="lf.bin")
    pow_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="pow.nc")

    for bin_path, pow_path in zip(bin_paths, pow_paths):
        run_bandpower_pipeline_on_file(bin_path, pow_path, cx_chans, sr_chans)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

# Run automated pipeline

### Segundo

In [11]:
run_bandpower_pipeline_on_condition(subject="Segundo", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 18000001
nChan: 385, nFileSamp: 18000001
17:46:44: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t23.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
nChan: 385, nFileSamp: 18000001
17:51:48: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t24.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
nChan: 385, nFileSamp: 18000001
17:57:13: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t25.imec0.lf.bin


### Valentino

In [10]:
run_bandpower_pipeline_on_condition(subject="Valentino", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:03:19: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/raw/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:07:56: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/raw/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:14:20: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/raw/2-20-2020_g0/2-20-2020_g0_imec0/2-20-2020_g0_t5.imec0.lf.bin


### Doppio

In [9]:
run_bandpower_pipeline_on_condition(subject="Doppio", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
17:26:55: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
17:29:40: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
17:36:00: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t5.imec0.lf.bin


### Alessandro

In [10]:
run_bandpower_pipeline_on_condition(subject="Alessandro", condition="extended-wake")

nChan: 385, nFileSamp: 9000051
nChan: 385, nFileSamp: 9000051
12:48:18: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
12:50:41: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 8127109
nChan: 385, nFileSamp: 8127109
12:53:35: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 8277840
nChan: 385, nFileSamp: 8277840
12:55:39: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD2_g0/8-25-2020_SD2_g0_imec0/8-25-2020_SD2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
12:58:25: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/r

In [12]:
run_bandpower_pipeline_on_condition(subject="Alessandro", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
12:31:50: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
12:34:04: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 9000051
nChan: 385, nFileSamp: 9000051
12:37:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
12:40:10: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t5.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
12:43:13: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25

### Eugene

In [9]:
run_bandpower_pipeline_on_condition(subject="Eugene", condition="extended-wake")

nChan: 385, nFileSamp: 800217
nChan: 385, nFileSamp: 800217
12:32:44: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs_g0/9.24.2020_SD_24hs_g0_imec0/9.24.2020_SD_24hs_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 2877424
nChan: 385, nFileSamp: 2877424
12:33:29: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs1_g0/9.24.2020_SD_24hs1_g0_imec0/9.24.2020_SD_24hs1_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 2430722
nChan: 385, nFileSamp: 2430722
12:34:10: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs2_g0/9.24.2020_SD_24hs2_g0_imec0/9.24.2020_SD_24hs2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 5735230
nChan: 385, nFileSamp: 5735230
12:35:36: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs3_g0/9.24.2020_SD_24hs3_g0_imec0/9.24.2020_SD_24hs3_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
12:37:59: Finished /Volumes/neu

In [13]:
run_bandpower_pipeline_on_condition(subject="Eugene", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
13:07:44: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
13:10:19: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
13:13:57: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
13:16:51: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
13:19:41: Finished /Volumes/neuropixel

KeyError: 'fileSizeBytes'

# Run pipeline piecemeal

In [11]:
%matplotlib widget

In [12]:
import matplotlib.pyplot as plt
from ecephys.plot import plot_spectrogram, plot_hypnogram_overlay
from ecephys.scoring import load_visbrain_hypnogram
from neurodsp.plts.time_series import plot_time_series
from ripple_detection.core import gaussian_smooth
smoothing_sigma = 2 # seconds

In [13]:
SUBJECT="Segundo"
CONDITION="REC-2-0"
START_TIME = 0
END_TIME = 60
bin_path = paths.get_datapath_from_csv(subject=SUBJECT, condition=CONDITION, data="lf.bin")

## Cortex

In [14]:
cx_chans = channel_groups.superficial_ctx[SUBJECT]
cx_sig = load_timeseries(bin_path, cx_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

nChan: 385, nFileSamp: 18000001


In [104]:
cx_spg = get_spectrogram(cx_sig)

In [105]:
cx_theta = _get_bandpower(cx_spg, THETA_RANGE, cx_chans)
cx_delta = _get_bandpower(cx_spg, DELTA_RANGE, cx_chans)

### Plot

In [36]:
plot_spectrogram(cx_spg.frequency, cx_spg.time, cx_spg, yscale='log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_delta.values, ax=ax, title="Cortical Delta", ylabel="Power")
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_theta.values, ax=ax, title="Cortical Theta", ylabel="Power")
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(cx_spg.time.values, cx_theta.values / cx_delta.values, title="Cortical theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Theta

In [106]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
sr_sig = load_timeseries(bin_path, sr_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

nChan: 385, nFileSamp: 18000001


In [107]:
sr_spg = get_spectrogram(sr_sig)

In [108]:
sr_theta = _get_bandpower(sr_spg, THETA_RANGE, sr_chans)
sr_delta = _get_bandpower(sr_spg, DELTA_RANGE, sr_chans)

### Plot

In [43]:
plot_spectrogram(sr_spg.frequency, sr_spg.time, sr_spg, yscale='log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_delta.values, title="SR Delta", ylabel="Power", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_theta.values, title="SR Theta", ylabel="Power", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, sr_theta.values / sr_delta.values, title="SR theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
_, ax = plt.subplots(figsize=(20, 3))
plot_time_series(sr_spg.time.values, np.log10(sr_theta.values) / np.log10(sr_delta.values), title="SR log(theta)/log(delta) ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(sr_spg.time.values, gaussian_smooth(sr_theta.values / sr_delta.values, smoothing_sigma, 1 / np.diff(sr_spg.time.values).mean()), title="Smoothed SR theta/delta ratio", ylabel="Ratio", ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Save

In [109]:
bandpower = xr.Dataset({'mpta_delta': cx_delta, 'mpta_theta': cx_theta, 'sr_delta': sr_delta, 'sr_theta': sr_theta})

In [110]:
bandpower.to_netcdf(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lfp_bandpower.nc"))

In [111]:
bandpower.close()